<a href="https://colab.research.google.com/github/fernandodeeke/can2025/blob/main/gaussian_elimination3_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <h2></h2></h1></center>
<center><h1>Numerical Analysis</h1></center>
<center><h2>2025/1</h2></center>
<center><h3>Fernando Deeke Sasse</h3></center>
<center><h3>CCT - UDESC</h3></center>
<center><h2>Gaussian Elimination - 3</h2></center>

### 1. Partial row pivoting

Let's consider the linear system $AX = B$ with

$$
A = \begin{bmatrix}
10^{-8} & 1 & 4 \\
1 & 1 & 4 \\
2 & 4 & 5
\end{bmatrix}\,,\,\,
B = \begin{bmatrix}
1 \\ 3\\ 4
\end{bmatrix}
$$

Let's solve this system using the GaussSolve algorithm, presented earlier:

In [18]:
import numpy as np

In [19]:
def GaussSolve(a,b):
    a = a.copy() # Preserve the input
    b = b.copy()
    n = len(a)
    x=np.zeros(n)
    for k in range(0,n-1):
        for i in range(k+1,n):
            lam = a[i,k]/a[k,k]
            a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
            b[i] = b[i] - lam*b[k]
    x[n-1]=b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (b[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return x

We define the matrices:

In [156]:
A = np.array([[1.e-15, 1.,4.,6.],[1., 6.,4.,7.],[2.,4.,5.,5.],[3.,5.,-1.,-3]])
B = np.array([1.,3.,4.,5.])

Let us test the algorithm:

In [157]:
X1 = GaussSolve(A,B)
X1

array([ 0.66613381,  0.2826087 ,  0.52173913, -0.22826087])

We may verify the result calculating the residue:

In [158]:
R1=A@X1-B
R1

array([ 0.        , -0.14908358, -0.06990628, -1.4255116 ])

We can note that there is a problem with this solution, as not all components are close to zero. Such a problem can be solved by an exchange of rows:

In [159]:
A2 = np.array([[1., 6.,4.,7.],[1.e-15, 1.,4.,6.],[2.,4.,5.,5.],[3.,5.,-1.,-3]])
B2 = np.array([3.,1.,4.,5.])

Let us try again:

In [160]:
X2 = GaussSolve(A2,B2)
X2

array([ 2.4, -0.2, -0.6,  0.6])

This is a completely different result.  Let us calculate the residue:

In [143]:
R2=A2@X2-B2
R2

array([ 0.00000000e+00, -4.44089210e-16,  1.77635684e-15, -8.88178420e-16])

which shows that the solution is now correct.

In what follows we may use the algorithm that performs elementary row operations:

In [75]:
def add_rows(A,k,i,j):
    "Adds k times the row j to row i in the matrix A"
    n=A.shape[0]
    E=np.eye(n)
    if i == j:
        E[i,i] = k+1
    else:
        E[i,j] = k
    return E@A

Another potential problem of the simple Gaussian elimination procedure is that when the row pivot is zero the algorithm will be stopped by dividing by zero at the time of calculating the multiplier.

### 2. Algorithm for partial row pivoting

To avoid the problems described above we can proceed systematically during the Gaussian elimination process by always passing up the line with the highest pivot. This is the so-called Gaussian elimination algorithm with partial row pivoting.  Below we describe a pseudo-code that describes this algorithm:

| Steps | |
| --: | :-- |
|  1  | For $i = 1$, ..., $n$ do Steps 2-4
|  2  | $\phantom{--}$ Find $p$, where $p$ is the largest number with $i\leq p\leq n$
|  3  | $\phantom{--}$ If $p \neq i$, then exchange row $i$ with row $p$
|  4  | $\phantom{--}$ For $j=i+1, ..., n$ do Steps 5-6
|  5  | $\phantom{----}$ set $m_{ji} = a_{ji}/a_{ii}$
|  6  | $\phantom{----}$ Perform $E_j = ( E_j - m_{ji}E_i)$
|  7  | Set $x_n = a_{n,n+1}/a_{nn}$
|  8  | For $i = n-1, ..., 1$ do Step 9
|  9  | $$ x_i=\left.\left(a_{i,n+1}-\sum_{j=i+1}a_{ij}x_j \right) \middle/ a_{ii} \right.$$

The following function performs partial row pivoting:

In [114]:
import numpy as np

def partial_pivot(a, b):
    a = a.copy()  # Preserve the input
    b = b.copy()
    n = a.shape[0]
    for i in range(n):
        # Find the row with the largest absolute value in the ith column
        max_row = i
        for j in range(i+1, n):
            if abs(a[j,i]) > abs(a[max_row,i]):
                max_row = j

        # Swap rows i and max_row in a and b
        a[[i, max_row], :] = a[[max_row, i], :]
        b[[i, max_row]] = b[[max_row, i]]

        # Eliminate the ith variable from the remaining rows
        for j in range(i+1, n):
            factor = a[j,i] / a[i,i]
            a[j,i:] -= factor * a[i,i:]
            b[j] -= factor * b[i]

    # Back-substitution
    x = np.zeros(n)  # Creates a 1D array of length n
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(a[i,i+1:], x[i+1:])) / a[i,i]  # Fixed 'A' to 'a'
    return x

In [115]:
a = np.array([[2, 1], [5, 3]], dtype=float)  # 2x2 matrix
b = np.array([5, 13], dtype=float)           # 1D array of length 2
x = partial_pivot(a, b)
x

array([2., 1.])

Let us test the algorithm:

In [166]:
A = np.array([[1.e-15, 1.,4.,6.],[1., 6.,4.,7.],[2.,4.,5.,5.],[3.,5.,-1.,-3]])
B = np.array([1.,3.,4.,5.])

In [162]:
X3 = np.array(partial_pivot(A,B))
X3

array([ 2.4, -0.2, -0.6,  0.6])

In [163]:
R3=A@X3-B
R3

array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -8.8817842e-16])

Partial row pivoting is implemented in the numpy solver:

In [167]:
X4 = np.linalg.solve(A,B)
print(X4)

[ 2.4 -0.2 -0.6  0.6]


### 3. Exercises

**1.** Let be the linear system $AX = B$ with

$$
A = \begin{bmatrix}
3.45 \times 10^{-10} & -4.34 & 5.98 \\
5.34 \times 10^{6} & 5.23 & 2.96 \\
4.39 \times 10^{10} & 4.32 & 15.32
\end{bmatrix}\,,\,\,
B = \begin{bmatrix}
1.98 \\ -8.39\\ 9.97
\end{bmatrix}
$$

(i) Try to solve the system step by step, using the Gaussian elimination process with partial row pivoting and calculate the residue. As a tool use the function

In [ ]:
def add_rows(A,k,i,j):
    "Add k times row j to row i"
    n=A.shape[0]
    E=np.eye(n)
    if i == j:
        E[i,i] = k+1
    else:
        E[i,j] = k
    return E@A

<br>
(ii) Solve the system using the partial_pivot function defined previously.
<br>
(iii) Solve the system using the numpy solver.

**2.** Solve problem 1 now with

$$
A = \begin{bmatrix}
0 & -4.34 & 5.98 & -4.32 \\
 3.45 & 333.23 & 2.96& 3.45 \\
3.97 & 4304.32 & 15.32& 432.203\\
433.97 & 4304.32 & 15.32& 432.203
\end{bmatrix}\,,\,\,
B = \begin{bmatrix}
1.98 \\ -8.39\\ 9.97 \\ 302.34
\end{bmatrix}
$$

**3.** Define a random 1000 x 1000 linear system. Use %timeit to measure the time necessary to solve this system using both the naîve Gauss elimination function GaussSolve and the partial_pivot defined in this notebook. What are your conclusions?

**4.** Build and implement an agorithm that performs partial row pivoting with scales.